<a href="https://colab.research.google.com/github/KleberKlaar/C/blob/main/Assistente_de_Plano_de_Aula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#INSTALAÇÕES PRINCIPAIS
!pip -q install google-genai
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7

In [6]:
# Importações

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings
import os
from google.colab import userdata
from google import genai

warnings.filterwarnings("ignore")

In [7]:
# Configura a API Key do Google Gemini

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [8]:
# Configura o cliente da SDK do Gemini

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [9]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [10]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [11]:
##########################################
# --- Agente 1: Requisitos --- #
##########################################
def requisitos(topico):
    requisitos = Agent(
        name="requisitos",
        model="gemini-2.5-flash-preview-04-17",
        tools=[google_search],
        instruction="""
        Você é um Agente de Requisitos para um sistema de criação de planos de aula.
        Sua tarefa é ler a descrição fornecida pelo usuário e extrair as seguintes informações específicas de forma clara e estruturada.
        Liste cada item encontrado. Se uma informação não for explicitamente mencionada, indique "Não especificado".
        Itens a extrair:
        - Tema Central
        - Nível de Ensino/Ano (Série)
        - Duração Estimada da Aula
        - Objetivos Preliminares ou Foco Principal (se o usuário mencionar)
        - Contexto da Turma ou Observações Adicionais (se o usuário mencionar algo sobre a turma ou necessidades especiais)
        Apresente o resultado como uma lista ou pares chave-valor.
        """
    )
    entrada_do_requisitos = f"Tópico: {topico}"

    saida_requisitos = call_agent(requisitos, entrada_do_requisitos)
    return saida_requisitos

In [19]:
################################################
# --- Agente 2: Referenciais Curriculares --- #
################################################
def referenciais_curriculares(topico, saida_requisitos):
    referenciais_curriculares = Agent(
        name="referenciais_curriculares",
        model="gemini-2.5-flash-preview-04-17",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um Integrador de Referenciais Curriculares para planos de aula.
        Você recebeu os requisitos básicos do plano (Tema, Nível de Ensino, etc.).
        Sua tarefa é identificar as diretrizes, habilidades e competências relevantes da BNCC (Base Nacional Comum Curricular), LDB (Lei de Diretrizes e Bases da Educação)
        e, se possível, de currículos estaduais/municipais comuns para este Tema e Nível de Ensino.
        Também é responsável por levantar referências bibliográficas para embasar as aulas.
        Use a ferramenta de busca (Google Search) para encontrar essas informações.
        Concentre-se nas habilidades e competências mais diretamente aplicáveis.
        Apresente os referenciais encontrados de forma clara, listando códigos (se houver) e
        descrições das habilidades/competências relevantes.
        """,
        tools=[google_search]
    )

    entrada_do_referenciais_curriculares = f"Tópico:{topico}\nLançamentos buscados: {saida_requisitos}"
    # Executa o agente
    saida_referenciais = call_agent(referenciais_curriculares, entrada_do_referenciais_curriculares)
    return saida_referenciais

In [13]:
################################################
# --- Agente 3: Elaborador --- #
################################################
def elaborador(topico, saida_referenciais):
    elaborador = Agent(
        name="elaborador",
        model="gemini-2.5-flash-preview-04-17",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um Elaborador de Plano de Aula Base.
        Você recebeu os requisitos do plano (Tema, Nível, Duração, etc.) e os referenciais curriculares aplicáveis (BNCC, LDB, etc.).
        Sua tarefa é rascunhar a estrutura inicial do plano de aula.
        Inclua as seguintes seções:
        - Título da Aula: Um título claro baseado no tema.
        - Objetivos de Aprendizagem: Escreva objetivos claros e mensuráveis para os alunos, alinhados com os referenciais curriculares fornecidos e o tema/nível.
        - Conteúdo Programático: Liste os principais tópicos ou conceitos que serão abordados na aula.
        - Etapas da Aula: Divida a aula em fases básicas (ex: Introdução/Aquecimento, Desenvolvimento, Conclusão/Fechamento), indicando brevemente o propósito de cada fase.
        Apresente esta estrutura de plano base de forma organizada.
        """,
        tools=[google_search]
    )

    entrada_do_elaborador = f"Tópico:{topico}\nLançamentos buscados: {saida_referenciais}"
    # Executa o agente
    saida_elaborador = call_agent(elaborador, entrada_do_elaborador)
    return saida_elaborador

In [14]:
################################################
# --- Agente 4: Metodologia --- #
################################################
def metodologia(topico, saida_elaborador):
    metodologia = Agent(
        name="metodologia",
        model="gemini-2.5-flash-preview-04-17",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um Detalhador de Planos de Aula. Você recebeu um rascunho de plano de aula base (com Título, Objetivos, Conteúdo e Etapas). Sua tarefa é adicionar detalhes práticos e pedagógicos a este rascunho.
        Para cada etapa da aula (Introdução, Desenvolvimento, Conclusão), descreva Sugestões de Atividades específicas que os alunos podem realizar.
        Sugira Metodologias de Ensino apropriadas para o tema, nível e atividades propostas (ex: aula expositiva dialogada, trabalho em grupo, debate, experimentação, uso de mídias).
        Liste os Recursos Didáticos e Materiais necessários para as atividades e metodologias sugeridas (ex: quadro branco, projetor, materiais de experimento, textos, vídeos, online tools).
        Certifique-se de que as sugestões de atividades, metodologias e recursos estejam alinhadas com os Objetivos e o Conteúdo do plano base. Use a ferramenta de busca se precisar de ideias.
        Apresente o plano base expandido com as novas seções e detalhes.
        """,
        tools=[google_search]
    )

    entrada_do_metodologia = f"Tópico:{topico}\nLançamentos buscados: {saida_elaborador}"
    # Executa o agente
    saida_metodologia = call_agent(metodologia, entrada_do_metodologia)
    return saida_metodologia

In [15]:
################################################
# --- Agente 5: Avaliador --- #
################################################
def avaliador(topico, saida_metodologia):
    avaliador = Agent(
        name="avaliador",
        model="gemini-2.5-flash-preview-04-17",
        # Inserir as instruções do Agente #################################################
        instruction="""
        Você é um Detalhador de Planos de Aula. Você recebeu um rascunho de plano de aula base (com Título, Objetivos, Conteúdo e Etapas). Sua tarefa é adicionar detalhes práticos e pedagógicos a este rascunho.
        Para cada etapa da aula (Introdução, Desenvolvimento, Conclusão), descreva Sugestões de Atividades específicas que os alunos podem realizar.
        Sugira Metodologias de Ensino apropriadas para o tema, nível e atividades propostas (ex: aula expositiva dialogada, trabalho em grupo, debate, experimentação, uso de mídias).
        Liste os Recursos Didáticos e Materiais necessários para as atividades e metodologias sugeridas (ex: quadro branco, projetor, materiais de experimento, textos, vídeos, online tools).
        Certifique-se de que as sugestões de atividades, metodologias e recursos estejam alinhadas com os Objetivos e o Conteúdo do plano base. Use a ferramenta de busca se precisar de ideias.
        Apresente o plano base expandido com as novas seções e detalhes.
        """,
        tools=[google_search]
    )

    entrada_do_avaliador = f"Tópico:{topico}\nLançamentos buscados: {saida_metodologia}"
    # Executa o agente
    saida_avaliador = call_agent(avaliador, entrada_do_avaliador)
    return saida_avaliador

In [16]:
################################################
# --- Agente 6: Coordenador --- #
################################################
def coordenador(topico, saida_avaliador):
    coordenador = Agent(
        name="coordenador",
        model="gemini-2.5-flash-preview-04-17",
        # Inserir as instruções do Agente #################################################
        instruction="""
        Você é um Coordenador Pedagógico experiente. Você recebeu um rascunho completo de plano de aula. Sua tarefa é revisar este plano de forma crítica e construtiva.
        Verifique a coerência e o alinhamento entre todas as partes: Os objetivos se encaixam nos referenciais? O conteúdo cobre os objetivos? As atividades realmente ajudam a aprender o conteúdo e alcançar os objetivos? A avaliação mede o que foi proposto?
        Avalie a viabilidade prática: O tempo (duração estimada) é realista para todas as atividades e conteúdo? Os recursos são comuns/acessíveis?
        Considere a adequação para o nível de ensino e o contexto da turma (se mencionado nos requisitos).
        Identifique pontos fortes e áreas que precisam de melhoria. Forneça observações pedagógicas e sugestões específicas para refinar o plano. Apresente suas observações e sugestões de forma clara, referenciando as partes do plano a que se aplicam. Você pode adicionar uma nova seção "Observações do Coordenador" ou integrar seus comentários diretamente no texto do plano, marcando-os claramente.
        Confira um por um dos códigos BNCC informados, se estão corretos e sugira a correção dos que estiverem errados.
        """,
        tools=[google_search]
    )

    entrada_do_coordenador = f"Tópico:{topico}\nLançamentos buscados: {saida_avaliador}"
    # Executa o agente
    saida_coordenador = call_agent(coordenador, entrada_do_coordenador)
    return saida_coordenador

In [17]:
################################################
# --- Agente 7: Final --- #
################################################
def final(topico, saida_coordenador):
    final = Agent(
        name="final",
        model="gemini-2.5-flash-preview-04-17",
        # Inserir as instruções do Agente #################################################
        instruction="""
        Você é o Agente Formatador Final de Planos de Aula. Você receberá um rascunho completo e revisado de um plano de aula. Este rascunho contém diversas informações geradas pelos agentes anteriores, e PODE incluir observações ou comentários de um coordenador pedagógico.

        Sua tarefa é:
        1. Ler e identificar todas as informações relevantes do plano: Título, Tema, Nível/Série, Duração, Requisitos Iniciais, Referenciais Curriculares (incluindo códigos e descrições da BNCC/LDB/etc.), Objetivos de Aprendizagem, Conteúdo Programático, Descrições das Etapas/Atividades, Metodologias, Materiais/Recursos, Sugestões de Avaliação.
        2. **IGNORAR COMPLETAMENTE** quaisquer observações, comentários, sugestões de melhoria ou seções adicionadas pelo "Coordenador Pedagógico" ou outro agente de revisão que não façam parte do conteúdo intrínseco do plano.
        3. **Organizar e formatar SOMENTE o conteúdo relevante do plano** no template exato e com os nomes de seção especificados abaixo.
        4. Garantir que os **códigos da BNCC (e de outros referenciais, se houver)** e suas descrições correspondentes, que foram identificados em etapas anteriores, sejam incluídos na seção "PARÂMETROS CURRICULARES".

        O plano final DEVE seguir ESTE TEMPLATE exato e conter APENAS as seções listadas abaixo, nesta ordem, com estes títulos:

        **TÍTULO**
        [Título claro da aula]

        **Turma/Seriação**
        [Nível de Ensino/Ano]

        **VISÃO GERAL E OBJETIVO**
        [Uma breve introdução sobre o tema da aula para esta turma, seguida por uma síntese do objetivo principal ou propósito geral da aula.]

        **PARÂMETROS CURRICULARES**
        [Liste aqui os códigos completos (ex: EF07CI09) e as descrições das habilidades e competências relevantes da BNCC, LDB ou outros referenciais, conforme identificados na pesquisa inicial.]

        **OBJETIVOS**
        [Liste os objetivos de aprendizagem específicos e mensuráveis para os alunos.]

        **CONTEÚDO**
        [Liste os tópicos principais ou conceitos que serão abordados na aula.]

        **DESENVOLVIMENTO**
        [Descreva as etapas da aula em detalhes, incluindo as atividades a serem realizadas em cada fase (Introdução, Desenvolvimento, Conclusão). Transforme as descrições das atividades em um fluxo contínuo ou lista clara de passos da aula.]

        **METODOLOGIA**
        [Descreva as principais abordagens e técnicas de ensino que serão utilizadas.]

        **MATERIAIS NECESSÁRIOS**
        [Liste todos os recursos didáticos e materiais que serão precisos para a aula.]

        **AVALIAÇÃO**
        [Descreva as formas sugeridas para avaliar o aprendizado dos alunos.]

        Certifique-se de preencher cada seção do template com as informações correspondentes extraídas do input, ignorando o conteúdo do revisor.
        """,
        tools=[google_search]
    )

    entrada_do_final = f"Tópico:{topico}\nLançamentos buscados: {saida_coordenador}"
    # Executa o agente
    saida_final = call_agent(final, entrada_do_final)
    return saida_final

In [22]:
print("Iniciando o Sistema de Criação de Planos de Aula")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TEMA sobre o qual você quer criar plano de aula: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de inserir um tema!")
else:
    print(f"Maravilha! Vamos, então, criar o plano de aula sobre {topico}")

    resultado = requisitos(topico)
    print("\n -- Resultados (Agente 1) -- \n")
    display(to_markdown(resultado))
    print("-------------------------------------------------------------")

    resultado = referenciais_curriculares(topico, resultado)
    print("\n -- Resultados (Agente 2) -- \n")
    display(to_markdown(resultado))
    print("-------------------------------------------------------------")

    resultado = elaborador(topico, resultado)
    print("\n -- Resultados (Agente 3) -- \n")
    display(to_markdown(resultado))
    print("-------------------------------------------------------------")

    resultado = metodologia(topico, resultado)
    print("\n -- Resultados (Agente 4) -- \n")
    display(to_markdown(resultado))
    print("-------------------------------------------------------------")

    resultado = avaliador(topico, resultado)
    print("\n -- Resultados (Agente 5) -- \n")
    display(to_markdown(resultado))
    print("-------------------------------------------------------------")

    resultado = coordenador(topico, resultado)
    print("\n -- Resultados (Agente 6) -- \n")
    display(to_markdown(resultado))
    print("-------------------------------------------------------------")

    resultado = final(topico, resultado)
    print("\n -- Resultados (Agente 7) -- \n")
    display(to_markdown(resultado))
    print("-------------------------------------------------------------")

Iniciando o Sistema de Criação de Planos de Aula
❓ Por favor, digite o TEMA sobre o qual você quer criar plano de aula: A origem de Roma para o sexto ano, com duração de uma aula.
Maravilha! Vamos, então, criar o plano de aula sobre A origem de Roma para o sexto ano, com duração de uma aula.

 -- Resultados (Agente 1) -- 



> Aqui está a extração das informações conforme solicitado:
> 
> *   **Tema Central:** A origem de Roma
> *   **Nível de Ensino/Ano (Série):** Sexto ano
> *   **Duração Estimada da Aula:** Uma aula
> *   **Objetivos Preliminares ou Foco Principal:** Não especificado
> *   **Contexto da Turma ou Observações Adicionais:** Não especificado


-------------------------------------------------------------

 -- Resultados (Agente 2) -- 



> Com base nos requisitos fornecidos (Tema: A origem de Roma, Nível de Ensino: Sexto ano, Duração: Uma aula), apresento os referenciais curriculares relevantes, com foco na BNCC e LDB, e a menção a currículos estaduais quando aplicável:
> 
> **BNCC (Base Nacional Comum Curricular)**
> 
> Para o sexto ano do Ensino Fundamental, o estudo da Antiguidade Clássica, incluindo a formação de Roma, é um tema previsto na BNCC. A habilidade mais diretamente relacionada ao tema "A origem de Roma" é:
> 
> *   **(EF06HI11)** Caracterizar o processo de formação da Roma Antiga e suas configurações sociais e políticas nos períodos monárquico e republicano.
> 
> Esta habilidade abrange a compreensão dos eventos e fatores que levaram ao surgimento de Roma, bem como suas primeiras estruturas sociais e políticas. Embora a habilidade mencione os períodos monárquico e republicano, para uma aula de duração limitada, o foco principal na origem (período monárquico e até o início da república) seria o mais adequado.
> 
> Outras habilidades da BNCC que podem ser tangencialmente relevantes, dependendo da abordagem da aula, incluem:
> 
> *   **(EF06HI09)** Discutir o conceito de Antiguidade Clássica, seu alcance e limite na tradição ocidental, assim como os impactos sobre outras sociedades e culturas. (Pode ser útil para contextualizar Roma dentro do estudo da Antiguidade Clássica).
> *   **(EF06HI02)** Identificar a gênese da produção do saber histórico e analisar o significado das fontes que originaram determinadas formas de registro em sociedades e épocas distintas. (Se a aula abordar as diferentes narrativas sobre a origem de Roma, como mitos e achados arqueológicos).
> 
> Além das habilidades específicas de História, as Competências Gerais da Educação Básica da BNCC também devem ser consideradas. Algumas relevantes para esta aula incluem:
> 
> *   **1. Conhecimento:** Valorizar e utilizar os conhecimentos historicamente construídos para entender a realidade.
> *   **2. Pensamento científico, crítico e criativo:** Exercitar a curiosidade intelectual e utilizar abordagens das ciências para investigar e analisar. (Ao analisar as diferentes versões sobre a origem de Roma).
> *   **3. Repertório cultural:** Valorizar as diversas manifestações artísticas e culturais. (Se a aula incluir a análise de representações artísticas ou literárias da origem de Roma).
> 
> **LDB (Lei de Diretrizes e Bases da Educação Nacional)**
> 
> A LDB estabelece as diretrizes e bases da educação brasileira. Em relação ao tema e nível de ensino, a LDB reforça a importância do estudo da história do Brasil e de outras civilizações. O Artigo 26 da LDB determina que os currículos da educação infantil, do ensino fundamental e do ensino médio devem ter uma base nacional comum, a ser complementada por uma parte diversificada. O estudo de Roma Antiga se encaixa na parte comum do currículo de História do Ensino Fundamental.
> 
> A LDB também destaca princípios como a garantia de padrão de qualidade e a valorização da experiência extraescolar, que podem ser considerados no planejamento da aula.
> 
> **Currículos Estaduais/Municipais Comuns**
> 
> Embora não tenha sido especificado um estado ou município, é importante notar que os currículos estaduais e municipais complementam a BNCC, detalhando os objetos de conhecimento e habilidades de acordo com as realidades locais. No entanto, a temática da origem de Roma para o sexto ano é universalmente presente nos currículos de História que seguem a BNCC para este nível, como exemplificado por referenciais do Paraná e materiais da Nova Escola alinhados à BNCC. O Currículo Paulista, por exemplo, em sua matriz curricular de História para o 6º ano, inclui explicitamente "As origens de Roma" como tópico a ser abordado.
> 
> Em suma, para uma aula sobre a origem de Roma para o sexto ano, o principal referencial é a habilidade **(EF06HI11)** da BNCC, que orienta o estudo do processo de formação e das primeiras configurações sociais e políticas de Roma Antiga. As competências gerais da BNCC, como conhecimento e pensamento crítico, também são fundamentais para guiar a abordagem pedagógica. A LDB fornece o embasamento legal para a inclusão deste tema no currículo do Ensino Fundamental. Currículos estaduais, como o Paulista, confirmam a relevância do tema para o 6º ano.


-------------------------------------------------------------

 -- Resultados (Agente 3) -- 



...

-------------------------------------------------------------

 -- Resultados (Agente 6) -- 



> Excelente rascunho! Como Coordenador Pedagógico, analisei seu plano detalhado sobre a origem de Roma para o 6º ano. Ele está muito bem estruturado, é criativo e busca engajar os alunos com diferentes linguagens.
> 
> Aqui estão minhas observações e sugestões para refinar ainda mais este excelente trabalho:
> 
> ---
> 
> **Observações do Coordenador Pedagógico**
> 
> **Avaliação Geral:** O plano é robusto, bem pensado e demonstra um excelente domínio do conteúdo e das metodologias para o 6º ano. A articulação entre o mito e a história, a localização geográfica e a introdução aos povos é fundamental para o tema e está bem contemplada. A estrutura em etapas com sugestões claras de atividades, metodologias e recursos facilita muito a execução pelo professor.
> 
> **1. Coerência e Alinhamento:**
> 
> *   **Objetivos x Conteúdo x Atividades:** Há um forte alinhamento. Os objetivos são claros e o conteúdo programático cobre precisamente o que foi proposto. As atividades em cada etapa estão diretamente ligadas ao conteúdo e visam ajudar os alunos a atingir os objetivos (compreender narrativas, localizar, identificar povos, reconhecer importância). O uso do mito como gancho inicial e a posterior distinção entre mito e história é uma estratégia pedagógica muito eficaz para esta faixa etária e alinhada com os objetivos.
> *   **Objetivos x BNCC:**
>     *   EF06HI11 ("Identificar e comparar as diferentes narrativas de fundação de sociedades antigas... e analisar os processos de formação dessas culturas e sociedades, com destaque para os povos que as originaram.") - **Correto.** Os objetivos e conteúdo abordam diretamente a narrativa de fundação (mito vs. história), o processo de formação (geografia, povos) e os povos originários.
>     *   EF06HI02 ("Identificar e analisar a importância dos mitos, ritos e festas para a transmissão de conhecimentos e culturas entre diferentes sociedades antigas.") - **Correto.** O foco no mito de Rômulo e Remo e a discussão sobre seu papel como narrativa fundadora se encaixam perfeitamente neste código.
>     *   Competências Gerais 1 e 2 - **Correto.** O estudo das origens de forma crítica (comparando mito e história) e a valorização do conhecimento histórico (CG1 e CG2) estão presentes.
> 
>     *   **Verificação dos Códigos BNCC:** Os códigos BNCC (EF06HI11 e EF06HI02) e as Competências Gerais (1 e 2) estão **corretamente** identificados e alinhados aos objetivos propostos. Não há necessidade de correção neste ponto.
> 
> **2. Viabilidade Prática:**
> 
> *   **Duração Estimada:** O plano propõe 50-60 minutos (considerando as somas das etapas). A distribuição (10-15 min Introdução, 30-35 min Desenvolvimento, 10-15 min Conclusão) é típica para uma aula expositiva/dialógica. No entanto, a seção de *Desenvolvimento* (30-35 min) está **bastante densa** para cobrir com alguma profundidade: localização geográfica (com explicação da importância), *terminar* o mito, *discutir* mito vs. história, *apresentar* os povos *e* suas influências, *e* ainda introduzir a formação histórica e o início da monarquia.
>     *   *Sugestão:* Este é o principal ponto de atenção em termos de viabilidade prática. É provável que o professor precise ser bastante ágil ou priorizar certos aspectos dentro do Desenvolvimento. Sugiro adicionar uma nota ao professor sobre a importância de gerenciar bem o tempo nesta etapa, focando nos pontos chave (a distinção mito/história e a importância da geografia e dos povos iniciais) e tratando a monarquia apenas como um gancho para a próxima aula, sem aprofundamento. Outra opção, se possível, seria considerar dividir este conteúdo em duas aulas, caso a profundidade desejada não caiba em uma única. Contudo, para *uma única* aula introdutória, o plano está adequado, apenas exigirá foco na execução.
> *   **Recursos:** Os recursos listados são comuns e acessíveis na maioria das escolas (projetor, mapa, quadro, caderno). O vídeo sobre o mito precisa ser cuidadosamente selecionado para a faixa etária e duração.
> 
> **3. Adequação para o Nível:**
> 
> *   O plano é altamente adequado para o 6º ano. A abordagem que inicia com o mito e depois apresenta o contexto histórico/geográfico funciona muito bem com a curiosidade e o modo de pensar dos alunos dessa idade. As atividades propostas (discussão, visualização, mapa, registro rápido) são variadas e condizentes com as capacidades e interesses do público.
> 
> **4. Pontos Fortes:**
> 
> *   Uso inteligente do mito como ponto de partida e gancho.
> *   Clareza na distinção entre mito e história, um conceito fundamental para a História.
> *   Valorização da geografia na explicação do desenvolvimento urbano.
> *   Variedade de metodologias e recursos que promovem diferentes formas de interação com o conteúdo.
> *   Etapa de conclusão eficaz para sintetizar e verificar a compreensão.
> *   Bom alinhamento com a BNCC.
> 
> **5. Áreas de Melhoria / Sugestões Específicas:**
> 
> *   **Gerenciamento do Tempo no Desenvolvimento:** Como mencionado em "Viabilidade Prática", a etapa de Desenvolvimento é ambiciosa para 30-35 minutos.
>     *   *Sugestão:* Incluir no plano, talvez como uma nota para o professor, a recomendação de priorizar a clareza da diferença entre mito e história e a relevância da geografia. Os povos e a monarquia podem ser apresentados de forma mais panorâmica, garantindo que os conceitos essenciais sobre a origem (mito vs. história, localização) sejam bem fixados.
> *   **Avaliação Mais Explícita:** Embora a conclusão preveja verificar a compreensão, o plano não tem uma seção explícita sobre avaliação.
>     *   *Sugestão:* Adicionar uma pequena seção "Avaliação" ou "Verificação da Aprendizagem" que descreva como o professor irá monitorar a compreensão dos alunos ao longo da aula (observação da participação nas discussões, respostas no quiz oral, análise dos registros rápidos no caderno) e o que será considerado para verificar se os objetivos foram atingidos. Isso formaliza o processo avaliativo já proposto implicitamente.
> *   **Diferenciação (Opcional):** Para tornar o plano ainda mais rico, poderiam ser adicionadas sugestões pontuais para lidar com diferentes ritmos de aprendizagem.
>     *   *Sugestão:* Na etapa de Conclusão/Registro, sugerir perguntas diferenciadas para o registro escrito ou desenho, por exemplo: "Para quem achou mais fácil: Por que é importante saber a história e o mito da origem de Roma?" ou "Para quem precisa de mais apoio: Desenhe a loba e os bebês *ou* mostre no mapa onde Roma fica."
> 
> ---
> 
> **Plano de Aula Detalhado: A Origem de Roma**
> 
> *   **Título da Aula:** A Fundação de Roma: Entre Mitos e Fatos Históricos
> 
> *   **Objetivos de Aprendizagem:**
>     *   Compreender as principais narrativas (mitológicas e históricas) sobre a origem da cidade de Roma. (Alinhado à EF06HI11 e EF06HI02)
>     *   Localizar geograficamente a região onde Roma foi fundada, identificando fatores importantes para seu desenvolvimento inicial. (Alinhado à EF06HI11)
>     *   Identificar, de forma introdutória, os povos que habitavam a região da Península Itálica na época da fundação de Roma. (Alinhado à EF06HI11)
>     *   Reconhecer a importância do estudo das origens para entender a história posterior de Roma. (Alinhado às Competências Gerais 1 e 2)
> 
> *   **Conteúdo Programático:**
>     *   Localização geográfica da Península Itálica e da cidade de Roma (Rio Tibre, Sete Colinas).
>     *   As diferentes versões sobre a origem de Roma: o mito de Rômulo e Remo.
>     *   Os primeiros habitantes da região (Latinos, Sabinos, Etruscos).
>     *   A fundação da cidade e o início do período monárquico (breve introdução).
> 
> *   **Etapas da Aula, Atividades, Metodologias e Recursos:**
> 
>     1.  **Introdução / Aquecimento (aprox. 10-15 minutos):**
>         *   **Propósito:** Despertar o interesse dos alunos sobre a Roma Antiga e introduzir o tema da origem.
>         *   **Sugestões de Atividades:**
>             *   Iniciar a aula exibindo imagens impactantes de monumentos romanos atuais (Coliseu, Panteão, Fórum Romano) e perguntar aos alunos o que eles sabem ou imaginam sobre essa civilização antiga. Anotar as respostas no quadro.
>             *   Apresentar a pergunta disparadora: "Toda grande cidade tem uma história de como começou. Como vocês acham que uma cidade tão importante como Roma surgiu?"
>             *   Contar de forma breve e cativante o início do mito de Rômulo e Remo, parando no momento em que os bebês são abandonados no rio Tibre, gerando curiosidade sobre o que acontece a seguir.
>         *   **Metodologias de Ensino:**
>             *   Roda de conversa inicial para levantamento de conhecimentos prévios.
>             *   Apresentação visual (imagens).
>             *   Narrativa oral para introduzir o mito.
>         *   **Recursos Didáticos e Materiais:**
>             *   Projetor ou televisão.
>             *   Imagens de monumentos romanos.
>             *   Quadro branco ou flipchart.
>             *   Canetas ou giz.
> 
>     2.  **Desenvolvimento (aprox. 30-35 minutos):**
>         *   **Propósito:** Apresentar os principais conteúdos sobre a origem de Roma, explorando tanto o mito quanto as evidências históricas e geográficas.
>         *   **Sugestões de Atividades:**
>             *   **Localização Geográfica:** Utilizar um mapa da Itália Antiga ou da Península Itálica para mostrar a localização de Roma. Destacar o Rio Tibre e as Sete Colinas, explicando como a geografia local influenciou o desenvolvimento da cidade (facilidade de defesa, acesso à água).
>             *   **O Mito de Rômulo e Remo:** Concluir a narrativa do mito de Rômulo e Remo, utilizando um vídeo curto ou lendo um trecho adaptado. Promover uma breve discussão: "O que vocês acharam dessa história? Parece real ou fantasia?"
>             *   **Mito vs. História:** Explicar a diferença entre mito (narrativa simbólica para explicar origens ou fenômenos, muitas vezes com elementos sobrenaturais) e história (estudo do passado baseado em evidências e documentos). Mencionar que o mito de Rômulo e Remo, embora famoso, é uma lenda, enquanto a história busca outras explicações.
>             *   **Primeiros Habitantes:** Apresentar os principais povos que habitavam a região da Península Itálica antes da fundação de Roma (Latinos, Sabinos, Etruscos), utilizando um mapa ou esquema visual. Explicar brevemente a influência desses povos na formação de Roma (principalmente os Etruscos).
>             *   **Fatos Históricos (Breve):** Mencionar que, historicamente, Roma se formou a partir da união de aldeias de pastores Latinos e Sabinos nas colinas próximas ao Rio Tibre, e que os Etruscos tiveram um papel importante no desenvolvimento inicial da cidade.
>             *   **Início da Monarquia (Breve):** Introduzir a ideia de que, após a fundação, Roma foi governada por reis.
>         *   **Metodologias de Ensino:**
>             *   Aula expositiva dialogada, incentivando a participação dos alunos.
>             *   Uso de recursos audiovisuais (vídeo, imagens).
>             *   Análise de mapas.
>             *   Discussão comparativa (mito vs. história).
>         *   **Recursos Didáticos e Materiais:**
>             *   Mapa da Península Itálica ou Itália Antiga.
>             *   Projetor ou televisão.
>             *   Vídeo curto sobre o mito de Rômulo e Remo (adaptado para a faixa etária).
>             *   Trecho adaptado do mito para leitura.
>             *   Imagens ou esquemas visuais dos povos da região.
>             *   Quadro branco ou flipchart.
>         *   ***Observação do Coordenador:*** *Esta etapa contém bastante conteúdo para 30-35 minutos. O professor deve priorizar a clareza na distinção entre mito e história e a compreensão da influência geográfica na formação de Roma. A apresentação dos povos e do início da monarquia pode ser mais concisa para garantir que os pontos principais sejam abordados sem pressa.*
> 
>     3.  **Conclusão / Fechamento (aprox. 10-15 minutos):**
>         *   **Propósito:** Sintetizar o aprendizado, verificar a compreensão dos alunos e amarrar os conteúdos, conectando a origem com a importância futura de Roma.
>         *   **Sugestões de Atividades:**
>             *   Retomar os objetivos da aula e perguntar aos alunos se eles sentem que os alcançaram.
>             *   Realizar um "Quiz Rápido Oral" com perguntas sobre a localização de Roma, o mito de Rômulo e Remo, e a diferença entre mito e história.
>             *   Propor uma atividade de registro rápido no caderno: pedir que escrevam uma frase sobre o que acharam mais interessante sobre a origem de Roma ou desenhem um elemento da aula (a loba, o mapa, um monumento).
>             *   Conectar a aula atual com as próximas, explicando que a partir dessa origem humilde, Roma se tornaria uma das civilizações mais importantes da história.
>         *   **Metodologias de Ensino:**
>             *   Revisão oral e interativa.
>             *   Registro individual.
>             *   Fechamento e projeção para aulas futuras.
>         *   **Recursos Didáticos e Materiais:**
>             *   Caderno dos alunos.
>             *   Lápis ou caneta.
>             *   Quadro branco ou flipchart para anotar ideias chave da revisão.
> 
>     ***Verificação da Aprendizagem:***
>     *   A verificação da aprendizagem ocorrerá ao longo da aula através da observação da participação dos alunos nas rodas de conversa e discussões.
>     *   Será consolidada na etapa de Conclusão/Fechamento, verificando as respostas dos alunos no "Quiz Rápido Oral" e analisando os registros escritos/desenhos no caderno para identificar se compreenderam a localização de Roma, as narrativas de origem (mito vs. história) e algum aspecto relevante sobre o tema.
> 
> ---
> 
> Este plano detalhado atende perfeitamente ao que foi solicitado, fornecendo um guia prático e completo para o professor executar a aula sobre a origem de Roma para o 6º ano. As observações e sugestões visam apenas otimizar a execução e a clareza em pontos chave. Parabéns pelo excelente trabalho!


-------------------------------------------------------------

 -- Resultados (Agente 7) -- 



> **TÍTULO**
> A Fundação de Roma: Entre Mitos e Fatos Históricos
> 
> **Turma/Seriação**
> 6º Ano
> 
> **VISÃO GERAL E OBJETIVO**
> Esta aula introduzirá os alunos do 6º ano ao estudo das origens da cidade de Roma, explorando as diferentes narrativas sobre sua fundação, desde o famoso mito de Rômulo e Remo até as evidências históricas e a importância da localização geográfica. O objetivo principal é que os alunos compreendam as principais histórias sobre a origem de Roma, saibam diferenciar mito e história, e reconheçam a importância da geografia e dos primeiros povos na formação da cidade.
> 
> **PARÂMETROS CURRICULARES**
> EF06HI11 - Identificar e comparar as diferentes narrativas de fundação de sociedades antigas (suméria, egípcia, cretense, micênica, fenícia, grega, romana) e analisar os processos de formação dessas culturas e sociedades, com destaque para os povos que as originaram.
> EF06HI02 - Identificar e analisar a importância dos mitos, ritos e festas para a transmissão de conhecimentos e culturas entre diferentes sociedades antigas.
> Competência Geral 1 - Conhecimento: Valorizar e utilizar os conhecimentos historicamente construídos sobre o mundo físico, social, cultural e digital para entender e explicar a realidade, continuar aprendendo e colaborar para a construção de uma sociedade justa, democrática e inclusiva.
> Competência Geral 2 - Pensamento científico, crítico e criativo: Exercitar a curiosidade intelectual e recorrer à abordagem própria das ciências, incluindo a investigação, a reflexão, a análise crítica, a imaginação e a criatividade, para investigar causas, elaborar e testar hipóteses, formular e resolver problemas e criar soluções (inclusive tecnológicas) com base nos conhecimentos das diferentes áreas.
> 
> **OBJETIVOS**
> *   Compreender as principais narrativas (mitológicas e históricas) sobre a origem da cidade de Roma.
> *   Localizar geograficamente a região onde Roma foi fundada, identificando fatores importantes para seu desenvolvimento inicial.
> *   Identificar, de forma introdutória, os povos que habitavam a região da Península Itálica na época da fundação de Roma.
> *   Reconhecer a importância do estudo das origens para entender a história posterior de Roma.
> 
> **CONTEÚDO**
> *   Localização geográfica da Península Itálica e da cidade de Roma (Rio Tibre, Sete Colinas).
> *   As diferentes versões sobre a origem de Roma: o mito de Rômulo e Remo.
> *   Os primeiros habitantes da região (Latinos, Sabinos, Etruscos).
> *   A fundação da cidade e o início do período monárquico (breve introdução).
> 
> **DESENVOLVIMENTO**
> A aula será dividida em três etapas principais:
> 
> 1.  **Introdução / Aquecimento (aprox. 10-15 minutos):**
>     *   Iniciar a aula exibindo imagens impactantes de monumentos romanos atuais (Coliseu, Panteão, Fórum Romano).
>     *   Perguntar aos alunos o que eles sabem ou imaginam sobre essa civilização antiga e anotar as respostas no quadro.
>     *   Apresentar a pergunta disparadora: "Toda grande cidade tem uma história de como começou. Como vocês acham que uma cidade tão importante como Roma surgiu?"
>     *   Contar de forma breve e cativante o início do mito de Rômulo e Remo, parando no momento em que os bebês são abandonados no rio Tibre para gerar curiosidade.
> 
> 2.  **Desenvolvimento (aprox. 30-35 minutos):**
>     *   **Localização Geográfica:** Utilizar um mapa da Itália Antiga ou da Península Itálica para mostrar a localização de Roma. Destacar o Rio Tibre e as Sete Colinas, explicando como a geografia local influenciou o desenvolvimento da cidade.
>     *   **O Mito de Rômulo e Remo:** Concluir a narrativa do mito de Rômulo e Remo, utilizando um vídeo curto ou lendo um trecho adaptado. Promover uma breve discussão: "O que vocês acharam dessa história? Parece real ou fantasia?"
>     *   **Mito vs. História:** Explicar a diferença entre mito e história. Mencionar que o mito de Rômulo e Remo é uma lenda, enquanto a história busca outras explicações baseadas em evidências.
>     *   **Primeiros Habitantes:** Apresentar os principais povos que habitavam a região antes da fundação de Roma (Latinos, Sabinos, Etruscos), utilizando um mapa ou esquema visual. Explicar brevemente a influência desses povos.
>     *   **Fatos Históricos (Breve):** Mencionar que, historicamente, Roma se formou a partir da união de aldeias e que os Etruscos tiveram papel importante.
>     *   **Início da Monarquia (Breve):** Introduzir a ideia de que, após a fundação, Roma foi governada por reis.
> 
> 3.  **Conclusão / Fechamento (aprox. 10-15 minutos):**
>     *   Retomar os objetivos da aula.
>     *   Realizar um "Quiz Rápido Oral" sobre a localização de Roma, o mito de Rômulo e Remo, e a diferença entre mito e história.
>     *   Propor uma atividade de registro rápido no caderno: escrever uma frase sobre o que acharam mais interessante ou desenhar um elemento da aula.
>     *   Conectar a aula atual com as próximas, explicando que Roma se tornaria uma das civilizações mais importantes.
> 
> **METODOLOGIA**
> *   Roda de conversa inicial para levantamento de conhecimentos prévios.
> *   Apresentação visual (imagens, vídeo).
> *   Narrativa oral para introduzir o mito.
> *   Aula expositiva dialogada.
> *   Análise de mapas.
> *   Discussão comparativa (mito vs. história).
> *   Revisão oral e interativa.
> *   Registro individual.
> 
> **MATERIAIS NECESSÁRIOS**
> *   Projetor ou televisão.
> *   Imagens de monumentos romanos.
> *   Quadro branco ou flipchart.
> *   Canetas ou giz.
> *   Mapa da Península Itálica ou Itália Antiga.
> *   Vídeo curto sobre o mito de Rômulo e Remo (adaptado).
> *   Trecho adaptado do mito para leitura.
> *   Imagens ou esquemas visuais dos povos da região.
> *   Caderno dos alunos.
> *   Lápis ou caneta.
> 
> **AVALIAÇÃO**
> A verificação da aprendizagem ocorrerá ao longo da aula através da observação da participação dos alunos nas rodas de conversa e discussões. Será consolidada na etapa de Conclusão/Fechamento, verificando as respostas dos alunos no "Quiz Rápido Oral" e analisando os registros escritos/desenhos no caderno para identificar se compreenderam a localização de Roma, as narrativas de origem (mito vs. história) e algum aspecto relevante sobre o tema.


-------------------------------------------------------------
